# Produce Hourly Shaped Hydro Profile for the Texas Interconnection

ERCOT: Electric Reliability Council of Texas

The actual generation by fuel type for each 15-minute settlement interval reported by ERCOT [here](http://www.ercot.com/gridinfo/generation/) in their ***Fuel Mix Report*** is used in this notebook. The monthly files have been concatenated and the timestamps have been converted to UTC.

In [1]:
import pandas as pd
from numpy.testing import assert_almost_equal

from powersimdata.scenario.scenario import Scenario
from powersimdata.input.grid import Grid

Create an instance of the current grid

In [2]:
grid = Grid(["Texas"])

Reading bus.csv
Reading plant.csv
Reading gencost.csv
Reading branch.csv
Reading dcline.csv
Reading sub.csv
Reading bus2sub.csv
Reading zone.csv


In [3]:
hydro_index = pd.date_range(start="2016-01-01 00:00:00", end="2016-12-31 23:00:00", freq='H')

In [4]:
hydro_v2 = pd.DataFrame(index = hydro_index, columns=grid.plant[grid.plant['type'] == 'hydro'].index)

In [5]:
filename = "../../data/ercot_hydro_2016.csv"
hydro_profile_ercot = pd.read_csv(filename, header=0, index_col=0, parse_dates=True, squeeze=True)

In [6]:
total_hydro_capacity = grid.plant[grid.plant["type"] == "hydro"]["Pmax"].sum()
for i in hydro_v2.columns:
    # We do normalize profiles now, hence it is independent of individual plant capacity.
    hydro_v2[i] = [g / total_hydro_capacity for g in hydro_profile_ercot]

In [7]:
hydro_v2.clip(0,1,inplace=True)

In [8]:
hydro_v2.to_csv('texas_hydro_v2_normalize.csv')

In [9]:
hydro_plant_num = len(grid.plant[grid.plant["type"] == "hydro"])
assert_almost_equal(
    hydro_v2.sum().sum()*total_hydro_capacity/hydro_plant_num, 
    hydro_profile_ercot.sum(), 
    decimal=5
)